# Exercício 2


## Sobre a base de dados

Neste laboratório você utilizará um dataset contendo informações sobre filmes e cinemas. 

**Collections:**
* comments
* movies
* sessions
* theaters
* users




Veja um exemplo de documento de um dos filmes da collection `movies` e entenda a estrutura/schema:

```python
{
	'_id': ObjectId('573a1390f29313caabcd4135'),
	'awards': {
		'nominations': 0,
		'text': '1 win.',
		'wins': 1
	},
	'cast': ['Charles Kayser', 'John Ott'],
	'countries': ['USA'],
	'directors': ['William K.L. Dickson'],
	'fullplot': 'A stationary camera looks at a large ...',
	'genres': ['Short'],
	'imdb': {
		'id': 5,
		'rating': 6.2,
		'votes': 1189
	},
	'lastupdated': '2015-08-26 00:03:50.133000000',
	'num_mflix_comments': 1,
	'plot': 'Three men hammer on an anvil and pass a bottle of beer around.',
	'rated': 'UNRATED',
	'released': datetime(1893, 5, 9, 0, 0),
	'runtime': 1,
	'title': 'Blacksmith Scene',
	'tomatoes': {
		'lastUpdated': datetime(2015, 6, 28, 18, 34, 9),
		'viewer': {
			'meter': 32,
			'numReviews': 184,
			'rating': 3.0
		},
	},
	'type': 'movie',
	'year': 1893,
}
```



## Usando Python

In [ ]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
warnings.filterwarnings('ignore')
mongoclient = MongoClient('localhost', 27017)
db = mongoclient.mflix
print(db)

In [ ]:
db.movies.find({}).count()

## Questões Exemplo

### [Questão Exemplo A]  Mostre um documento qualquer da coleção movies

In [ ]:
result = db.movies.find({}).limit(1)

pprint( list(result) )

### [Questão Exemplo B]  Encontre os diretores do filme "Inglourious Basterds"

In [ ]:
result = db.movies.find({"title": "Inglourious Basterds"}, {"_id":0,"directors": 1})

list(result)

### [Questão 1] Mostre o ano de lançamento de todos os filme que começam com "One Night"

In [ ]:
result = db.movies.find({'title':{'$regex':'^One Night'}},
                        {'title':1,'year':1, '_id':0}
                       )
list(result)

### [Questão 2] Quantos filmes o diretor Quentin Tarantino dirigiu?

<!--
query = { 
         "directors": "Quentin Tarantino"
}

projection = { 
               "title": 1,
               "_id": 0   
}


result = db.movies.find(query, projection).distinct("title")
list(result)
-->

### [Questão 3] Quantos filmes do gênero "Drama" o diretor Quentin Tarantino dirigiu entre 1990 e 2005?


<!--
query = { 
         "directors": "Quentin Tarantino",
         "genres" : "Drama",
           "$and" : [ 
                     { "year": {"$gte":1990}},
                     { "year": {"$gte":1990}}
                    ]
}

projection = { 
               "title": 1,
               "genres":1, 
               "year": 1,
               "_id": 0
}


result = db.movies.find(query, projection)
list(result)
-->

### [Questão 4] Qual filme do diretor Quentin Tarantino teve mais prêmios?

Dica: Sort em Pymongo https://www.w3schools.com/python/python_mongodb_sort.asp

<!-- 
query = { 
         "directors": "Quentin Tarantino"
}

projection = { 
               "title": 1,
               "awards.wins":1,
               "_id": 0   
}


result = db.movies.find(query, projection).sort("awards.wins",-1)
list(result)
-->

### [Questão Extra] Quais documentos (filmes) possuem mesmo nome e quantas vezes cada um se repete?

<!--

-->

In [ ]:
pipeline = [
            {"$group": {"_id": "$title", "count": {"$sum": 1}}},
            {"$match": {"count":{"$gt":1}}},
            {"$sort":  {"count": -1}}
]


result = db.movies.aggregate(pipeline)
list(result)

### [Questão Extra] Quantos filmes cada diretor participou da direção? (ignore filmes repetidos)

In [ ]:
pipeline = [
            {"$unwind": "$directors"},
            {"$group": {"_id": "$directors", "count": {"$sum": 1}}},
            {"$sort":  {"count": -1}}  
]

result = db.movies.aggregate(pipeline)
list(result)

### [Questão Extra] Qual a somatória de prêmios recebidos pelos filmes para cada diretor? (ignore filmes repetidos)

In [ ]:
pipeline = [
            {"$unwind": "$directors"},
            {"$group": {"_id": "$directors", "count": {"$sum": "$awards.wins"}}},
            {"$sort":  {"count": -1}}
    
]


result = db.movies.aggregate(pipeline)
list(result)